<a href="https://colab.research.google.com/github/patrickdevv/FIAP_Fase03_Desafio/blob/main/Desafio03_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-mwhiwjtr/unsloth_e34e6aba436d478dafa55611417315dd
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-mwhiwjtr/unsloth_e34e6aba436d478dafa55611417315dd
  Resolved https://github.com/unslothai/unsloth.git to commit 85f1fa096afde5efe2fb8521d8ceec8d13a00715
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 11.2 MB/s eta 0:00:00
   ━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.13.0
    Uninstalling trl-0.13.0:
      Successfully uninstalled trl-0.13.0


#Tratamento do arquivo

In [3]:
import pandas as pd
import json

## Abre arquivo com dados iniciais

In [4]:

# Caminho do arquivo JSON
caminho_arquivo = '/content/drive/MyDrive/Colab Notebooks/desafio03/trn.json'

# Abrir o arquivo e ler linha por linha
with open(caminho_arquivo, 'r') as f:
    data = [json.loads(line) for line in f]

# Converter em DataFrame
df = pd.DataFrame(data)

# Visualizar o DataFrame
print(df.head())

          uid                                              title  \
0  0000031909                        Girls Ballet Tutu Neon Pink   
1  0000032034                           Adult Ballet Tutu Yellow   
2  0000913154  The Way Things Work: An Illustrated Encycloped...   
3  0001360000                                      Mog's Kittens   
4  0001381245                              Misty of Chincoteague   

                                             content  \
0  High quality 3 layer ballet tutu. 12 inches in...   
1                                                      
2                                                      
3  Judith Kerr&#8217;s best&#8211;selling adventu...   
4                                                      

                                          target_ind  \
0  [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...   
1  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 16, 33, 36, 37,...   
2                [116, 117, 118, 119, 120, 121, 122]   
3                          [14

## Filtrar o arquivo deixando somente as colunas Title e Content

In [5]:
# Filtrar apenas as colunas necessárias
df_filtrado = df[['title', 'content']]

# Nome do novo arquivo
nome_arquivo = '/content/drive/MyDrive/Colab Notebooks/desafio03/trn_filtrado.json'

df_filtrado.to_json(nome_arquivo, orient='records', lines=True)

print(f"Arquivo JSON filtrado salvo em: {nome_arquivo}")


Arquivo JSON filtrado salvo em: /content/drive/MyDrive/Colab Notebooks/desafio03/trn_filtrado.json


In [6]:
# Verificar informações gerais do DataFrame
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2248619 entries, 0 to 2248618
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   title    object
 1   content  object
dtypes: object(2)
memory usage: 34.3+ MB


In [7]:
# Visualizar o DataFrame
print(df_filtrado.head())
print(df_filtrado.count())

                                               title  \
0                        Girls Ballet Tutu Neon Pink   
1                           Adult Ballet Tutu Yellow   
2  The Way Things Work: An Illustrated Encycloped...   
3                                      Mog's Kittens   
4                              Misty of Chincoteague   

                                             content  
0  High quality 3 layer ballet tutu. 12 inches in...  
1                                                     
2                                                     
3  Judith Kerr&#8217;s best&#8211;selling adventu...  
4                                                     
title      2248619
content    2248619
dtype: int64


In [8]:
# Remover registros onde 'content' está vazio ou contém apenas espaços
df_filtrado_limpo = df_filtrado[df_filtrado['content'].str.strip() != '']

# Substituir " por ' na coluna 'content'
df_filtrado_limpo['content'] = df_filtrado_limpo['content'].apply(lambda x: x.replace('"', "'"))

# Substituir todas as ocorrências de \" por ' em 'content'
df_filtrado_limpo['content'] = df_filtrado_limpo['content'].apply(lambda x: x.replace('\\"', "'"))

# Verificar o resultado
print(df_filtrado_limpo.head())

<ipython-input-8-53f250acb3d7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado_limpo['content'] = df_filtrado_limpo['content'].apply(lambda x: x.replace('"', "'"))


                          title  \
0   Girls Ballet Tutu Neon Pink   
3                 Mog's Kittens   
7   Girls Ballet Tutu Neon Blue   
12                  The Prophet   
13    Rightly Dividing the Word   

                                              content  
0   High quality 3 layer ballet tutu. 12 inches in...  
3   Judith Kerr&#8217;s best&#8211;selling adventu...  
7   Dance tutu for girls ages 2-8 years. Perfect f...  
12  In a distant, timeless place, a mysterious pro...  
13         --This text refers to thePaperbackedition.  


<ipython-input-8-53f250acb3d7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado_limpo['content'] = df_filtrado_limpo['content'].apply(lambda x: x.replace('\\"', "'"))


In [9]:
print(df_filtrado_limpo.count())

title      1498718
content    1498718
dtype: int64


## Criar json para utilização em treinamento do LLM

### Grava arquivo para teste

In [10]:
import json

# Parte inicial e final para o prompt
parte_inicial_question = "[|Question|] What is the description for the product '"
parte_final_question = "'? [|eQuestion|]"
parte_inicial_answer = "[|Answer|] '"
parte_final_answer = "' [|eAnswer|]"

# Construir a estrutura desejada
novo_json = [
    {
        "prompt": f"{parte_inicial_question}{row['title']}{parte_final_question}",
        "completion": f"{parte_inicial_answer}{row['content']}{parte_final_answer}"
    }
    for _, row in df_filtrado_limpo.iterrows()
]

# Salvar o novo JSON estruturado
caminho_saida = '/content/drive/MyDrive/Colab Notebooks/desafio03/trn_test.json'
with open(caminho_saida, 'w') as f:
    json.dump(novo_json, f, ensure_ascii=False, indent=4)

print(f"Novo JSON formatado salvo em: {caminho_saida}")


Novo JSON formatado salvo em: /content/drive/MyDrive/Colab Notebooks/desafio03/trn_test.json


In [11]:
novo_json[0]

{'prompt': "[|Question|] What is the description for the product 'Girls Ballet Tutu Neon Pink'? [|eQuestion|]",
 'completion': "[|Answer|] 'High quality 3 layer ballet tutu. 12 inches in length' [|eAnswer|]"}

## Teste inicial antes do treinamento

Carrega arquivo de teste

In [12]:
import json
# Caminho do arquivo JSON
caminho_arquivo_test = '/content/drive/MyDrive/Colab Notebooks/desafio03/trn_test.json'

# Abrir o arquivo e ler linha por linha
with open(caminho_arquivo_test, 'r') as f:
     data = json.load(f)

# Converter em DataFrame
df_preparado_test = pd.DataFrame(data)

### Carregamento do modelo Llama3-8b

In [13]:
!pip install -U bitsandbytes
!pip install -U transformers
!pip install -U accelerate
!pip install --upgrade bitsandbytes
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 31.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.1.1
    Uninstalling accelerate-1.1.1:
      Successfully uninstalled accelerate-1.1.1


In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Configuração para quantização em 4 bits
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,  # Use 4-bit quantization
    llm_int8_threshold=6.0,  # Threshold padrão para balancear desempenho e memória
    llm_int8_has_fp16_weight=True  # Garante compatibilidade com FP16
)

# Nome do checkpoint do modelo
checkpoint = "unsloth/llama-3-8b-bnb-4bit"

# Carregar o tokenizador
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Carregar o modelo com configuração correta
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    device_map="auto",  # Distribuição automática nas GPUs disponíveis
    trust_remote_code=True,  # Aceitar código remoto do repositório do modelo
    quantization_config=bnb_config  # Configuração de quantização
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

In [15]:

# Função para extrair texto entre os delimitadores
def extrair_texto(prompt):
    print(prompt)
    if pd.isnull(prompt):  # Verificar se o valor é nulo
        return None
    match = re.search(r'\[\|Question\|\](.*?)\[\|eQuestion\|\]', prompt)
    if match:
        return match.group(1).strip()  # Retorna o texto extraído sem espaços em branco
    return None  # Retorna None se não houver correspondência

print(extrair_texto(df_preparado_test['prompt'][0]))



[|Question|] What is the description for the product 'Girls Ballet Tutu Neon Pink'? [|eQuestion|]
What is the description for the product 'Girls Ballet Tutu Neon Pink'?


In [16]:
# Testar com algumas amostras do dataset

from transformers import pipeline

# Função para geração de texto com o modelo base
def test_foundation_model(prompt, model, tokenizer, max_length=100):
    text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
    generated = text_generator(prompt, max_length=max_length, num_return_sequences=1)
    return generated[0]["generated_text"]


prompt = extrair_texto(df_preparado_test['prompt'][0])


for _, row in df_preparado_test.head(1).iterrows():  # Testar apenas o primeiro prompt
    #prompt= extrair_texto(row['prompt'])
    print("### Prompt:")
    print(prompt)
    print("### Resposta do Modelo Base:")
    print(test_foundation_model(prompt, model, tokenizer))
    print("=" * 80)

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[|Question|] What is the description for the product 'Girls Ballet Tutu Neon Pink'? [|eQuestion|]
### Prompt:
What is the description for the product 'Girls Ballet Tutu Neon Pink'?
### Resposta do Modelo Base:
What is the description for the product 'Girls Ballet Tutu Neon Pink'? 'Girls Ballet Tutu Neon Pink' is a product that is manufactured by the company 'Ballet Pink'. It is a type of tutu that is made from a material called 'Neon Pink'. It is a product that is used by ballet dancers.


#Prepara início do treinamento

In [17]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/desafio03/trn_test.json"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Colab Notebooks/desafio03/trn_test_output.json"
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [18]:
def format_dataset_into_model_input(data):
    def separate_text(full_text):
        question_start = full_text.find("[|Question|]") + len("[|Question|]")
        question_end = full_text.find("[|eQuestion|]")
        answer_start = full_text.find("[|Answer|]") + len("[|Answer|]")
        answer_end = full_text.find("[|eAnswer|]")

        # Extrair partes relevantes do texto
        instruction = "Answer the questions objectively"  # Primeira linha como instrução
        input_text = full_text[question_start:question_end].strip()
        response = full_text[answer_start:answer_end].strip()
        #print('instruction: ', instruction)
        #print('input_text: ', input_text)
        #print('response: ', response)
        return instruction, input_text, response

    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Iterar diretamente sobre a lista de dados
    for item in data:
        prompt = item['prompt']  # Extrair o campo 'prompt' de cada item
        completion = item['completion']  # Extrair o campo 'completion' de cada item

        # Separar os textos em partes relevantes
        instruction, input_text, response = separate_text(prompt + "\n" + completion)
        instructions.append(instruction)
        inputs.append(input_text)
        outputs.append(response)

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON
    OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Colab Notebooks/desafio03/formatted_dataset.json"
    with open(OUTPUT_PATH_DATASET, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")


In [19]:


format_dataset_into_model_input(novo_json)


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Dataset salvo em /content/drive/MyDrive/Colab Notebooks/desafio03/formatted_dataset.json
==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.47.0.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [20]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.12.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [21]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Colab Notebooks/desafio03/formatted_dataset.json"

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1498718 [00:00<?, ? examples/s]

In [22]:
dataset[1]

{'instruction': 'Answer the questions objectively',
 'input': "What is the description for the product 'Mog's Kittens'?",
 'output': "'Judith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books&#8212;with their bright, simple pictures, easy text, and hand&#8211;friendly formats&#8212;are just the thing to delight the very young. Ages 6 months&#8211;2 years.'",
 'text': "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nAnswer the questions objectively\n\n### Input:\nWhat is the description for the product 'Mog's Kittens'?\n\n### Response:\n'Judith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 

In [23]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/1498718 [00:00<?, ? examples/s]

In [24]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,498,718 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,11.691000
2,12.247800
3,12.319800
4,11.611500
5,9.972100
6,10.472700
7,8.834300
8,8.201200
9,8.291300
10,6.967100


In [25]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Answer this question",
        " What is the description for the product 'Girls Ballet Tutu Neon Pink'?", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True, temperature=0.2)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nAnswer this question\n\n### Input:\n What is the description for the product 'Girls Ballet Tutu Neon Pink'?\n\n### Response:\n'Neon Pink Ballet Tutu'<|end_of_text|>"]

In [29]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Answer this question",
        " What is the description for the product 'Girls Ballet Tutu Neon Pink'?", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128, temperature=0.2)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer this question

### Input:
 What is the description for the product 'Girls Ballet Tutu Neon Pink'?

### Response:
'Girls Ballet Tutu Neon Pink'<|end_of_text|>


In [27]:
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/desafio03/lora_model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/desafio03/lora_model")


('/content/drive/MyDrive/Colab Notebooks/desafio03/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/desafio03/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/desafio03/lora_model/tokenizer.json')